# Steup

## Dependencies

This notebook uses unreleased features, so intall Raster Vision from the master branch:

In [ ]:
%pip install git+https://github.com/azavea/raster-vision.git

In [ ]:
!git clone https://github.com/azavea/raster-vision.git --depth=1 && \
    pip install ./raster-vision/rastervision_aws_sagemaker

Install other dependencies:

In [ ]:
%pip install pystac_client==0.7.5

---

## AWS account

You will need access to the AWS account `#279682201306 | infrastructure+azaveard@element84.com` to be able to run this notebook.

---

In [ ]:
from typing import Any, Dict, List

from stactask import Task
from rastervision.aws_sagemaker import AWSSageMakerRunner

class RVPredTask(Task):
    name = 'rv-pred-task'
    description = 'Run prediction on an item using Raster Vision'

    def validate(self, payload: Dict[str, Any]) -> bool:
        return len(self.items) == 1

    def process(self, **kwargs: Any) -> List[Dict[str, Any]]:
        item = self.items[0]

        params = item['process']
        model_bundle_uri = params['model_bundle_uri']
        scene_config_uri = params['scene_config_uri']

        cmd = [
            'rastervision', 'predict_scene', model_bundle_uri, scene_config_uri]
        
        runner = AWSSageMakerRunner()
        runner.run_command(
            cmd=cmd,
            role='AmazonSageMakerExecutionRole',
            image_uri='279682201306.dkr.ecr.us-east-1.amazonaws.com/raster-vision-ahassan',
            instance_type='ml.p3.2xlarge',
            job_name='stac-pred',
        )

        return []

In [ ]:
payload = {}
task = RVPredTask(payload)